In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
# hyperparameters
batch_size = 128 # Increased from 16 -- batch_size = 16 ,,, how many independent sequences will we process in parallel? 
block_size = 256 # Increasing from 32 -- block_size = 32 ,,, what is the maximum context length for predictions?
max_iters = 2000
eval_interval = 100
learning_rate = 1e-3
weight_decay = 1e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 512       # Increased from 64 -- n_embd = 64
n_head = 8         # Increased from 4 -- n_head = 4
n_layer = 8        # Increased from 4 -- n_layer = 4
dropout = 0.2      # You might also consider adjusting dropout -- dropout = 0.0

In [3]:
torch.manual_seed(1337)

In [4]:
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('../input/text-datasets/tiny-TagorePoems.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [5]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)

In [6]:
# chars, vocab_size

In [7]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [8]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [9]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [10]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [11]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

In [12]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [13]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [14]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [15]:
# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [16]:
model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

25.497755 M parameters


In [17]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 5.1934, val loss 5.1998
step 100: train loss 2.7600, val loss 3.7140
step 200: train loss 2.6588, val loss 3.3084
step 300: train loss 2.6233, val loss 3.2181
step 400: train loss 2.4996, val loss 3.0278
step 500: train loss 2.2945, val loss 2.9169
step 600: train loss 2.1282, val loss 2.7562
step 700: train loss 2.0214, val loss 2.7526
step 800: train loss 1.9370, val loss 2.6883
step 900: train loss 1.8698, val loss 2.6589
step 1000: train loss 1.8116, val loss 2.6755
step 1100: train loss 1.7507, val loss 2.5948
step 1200: train loss 1.7001, val loss 2.6043
step 1300: train loss 1.6426, val loss 2.5986
step 1400: train loss 1.5911, val loss 2.6117
step 1500: train loss 1.5345, val loss 2.6117
step 1600: train loss 1.4861, val loss 2.6342
step 1700: train loss 1.4300, val loss 2.7016
step 1800: train loss 1.3666, val loss 2.6766
step 1900: train loss 1.3105, val loss 2.7599
step 1999: train loss 1.2433, val loss 2.7405


In [18]:
# After the training loop finishes, save the model:
torch.save(model.state_dict(), '../working/babyGPT-bn-25M-model-v0.pth')
print("Trained model saved to model.pth")

Trained model saved to model.pth


In [19]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=1000)[0].tolist()))


তেমার আনন্দে কে নক্ষত্রে 
অবসাদ কারো তোমারে সুর 
বাঁধয়া নাই দায় আপনারে আপন ন৭াভৈরব। 
করো সৌঁদন জীবন 
সেথা করো নাহি স্থাপ্পাঁরত 
সমদদ্রের প্রত্যুষগে 
একাকী যাঁর রান্র অস্তাঁশখরে 
ছাঁদয়া মৌন দিল তাহা তব না। 


৩৩ 
“বাধির অবসন্ত-অবলমস্থাপিয়া 


নীতের আবর্ভষেকভ' দয়া করেছে অনন্ত সনাদে তবু__ 
অনল্ত চলেছে পল্পবর্বন, 

ইতিহাসের মধ্য অস্ফুট হতে গৌণ সন্দুক, 
কোথায় ক্ষুধা তব্‌গায় অনন্ত পরশ ॥ 


 নভাময়া, অনল-মাদরে 
বাঁধা মা হোতে ॥ 
অক্ষম সদ্য়ে যাঁত জবালুশ্রতী, 
নন্দশ যেন নিজ্কমা জ্ঞান স্তব্ধ বাদ্ধে, ৪৫ 


অতীতের গোৌরবেদ। সম্পমাদর্ধান স্বাদ 
চন্দ্রস্নেহাত হোতে বসন বস্ত পারবে যে, 
হেরিণ সিংহাসন রবে জলি তার ॥ &amp;১৩ 


যে চ ত পিতা মোশে পাপারচয় 
কাঁবমোশ আপ কর্ষণ সে তান আত। 
দৈবা কশীর্ধত ছলো কবি মা কর্তবে। 
কাজে, নুপসা এ বন, 
খতু সত্যের দান যাবদেশে হবে 'মশ্বর+, 
লক্ষ ত্বাঁরি শঙ্কা মাঝারে 
সে যাবে বাজয়া চুকে, 
তখন বুক ভাঁঙ মোর কাছে সন্দেহেরে 
তেমনি অঙ্গে অতাঁতেরে অনিদেহ, 
আম ততুভঙ্গে 'গারুণ 
পদৃচ্টের ভূমি তত্যজয়ে বাজায়েছ 
গানেরেখার আনন্দে। 
তুমি তো 'দিয়ো না। 
একটি দেখো, নে সেই সুন্দর, দু